In [4]:
# import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

import h3
import math

import datatable as dt

In [4]:
ls data

ais_sample_baltic.csv  ports_baltic.csv


In [5]:
%%time

ports0 = pd.read_csv('data/ports_baltic.csv')
ports0.rename( columns = { 
        'lat' : 'lat_port',
        'lon' : 'lon_port',
        'country_code' : 'iso2',
    }, inplace=True )

ports0['h3_5'] = ports0.apply(lambda row: h3.geo_to_h3( lat = row['lat_port'], lng = row['lon_port'], resolution = 5), axis = 1)
ports0['h3_5_hexring'] = ports0.apply(lambda row: h3.hex_ring( row['h3_5'], k = 1), axis = 1)
ports0 = ports0.explode('h3_5_hexring').reset_index()

dt.Frame(ports0)

CPU times: user 14.2 ms, sys: 10.2 ms, total: 24.4 ms
Wall time: 26 ms


,index,name,port_id,lat_port,lon_port,iso2,h3_5,h3_5_hexring
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,851104d7fffffff
1,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,8511226ffffffff
2,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,85110493fffffff
3,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,85110483fffffff
4,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,85112267fffffff
5,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,8511048bfffffff
6,1,UST-LUGA,8214872,59.683,28.434,ru,85113103fffffff,8511310bfffffff
7,1,UST-LUGA,8214872,59.683,28.434,ru,85113103fffffff,85113117fffffff
8,1,UST-LUGA,8214872,59.683,28.434,ru,85113103fffffff,8511310ffffffff


In [6]:
ports2 = ports0.copy()
ports2['h3_5_hexring'] = ports2['h3_5']
ports2 = ports2.drop_duplicates(subset=['name','port_id','lat_port','lon_port','iso2','h3_5','h3_5_hexring'], keep = 'last').reset_index(drop = True) 
ports2 = ports2.sort_values(by=['name','port_id'], ascending=True)
ports = pd.concat([ports2, ports0]).sort_values(by=['name','port_id'], ascending=True)

ports['h3_5'] = ports['h3_5_hexring']
ports = ports[['name','port_id','lat_port','lon_port','iso2','h3_5']]
dt.Frame(ports)

,name,port_id,lat_port,lon_port,iso2,h3_5
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪
0,Agnesberg,2321,57.7856,12.0076,se,851f25abfffffff
1,Agnesberg,2321,57.7856,12.0076,se,851f2517fffffff
2,Agnesberg,2321,57.7856,12.0076,se,851f25a3fffffff
3,Agnesberg,2321,57.7856,12.0076,se,851f2533fffffff
4,Agnesberg,2321,57.7856,12.0076,se,851f2507fffffff
5,Agnesberg,2321,57.7856,12.0076,se,851f25affffffff
6,Agnesberg,2321,57.7856,12.0076,se,851f25bbfffffff
7,BERGKVARA,9238,56.3832,16.0831,se,851f28affffffff
8,BERGKVARA,9238,56.3832,16.0831,se,851f2833fffffff


In [7]:
%%time

# CPU times: user 21.2 s, sys: 4.15 s, total: 25.4 s
# Wall time: 25.6 s

ais = pd.read_csv('data/ais_sample_baltic.csv')
ais

<timed exec>:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 21.3 s, sys: 4.02 s, total: 25.4 s
Wall time: 25.6 s


,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,vessel_type,vessel_max_speed,draft,power,dwt,iceclass
0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14185041,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA
14185042,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA
14185043,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA
14185044,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA


In [8]:
%%time

# CPU times: user 48.2 s, sys: 2.02 s, total: 50.2 s
# Wall time: 50.2 s

ais = ais.drop_duplicates(subset=['mmsi', 'timestamp'], keep = 'last').reset_index(drop = True)
ais = ais.sort_values(by=['mmsi','timestamp'], ascending=True)
ais

CPU times: user 47.3 s, sys: 2.26 s, total: 49.6 s
Wall time: 49.5 s


,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,vessel_type,vessel_max_speed,draft,power,dwt,iceclass
0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183139,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA
14183140,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA
14183141,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA
14183142,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA


In [9]:
%%time

# CPU times: user 3min 48s, sys: 9.36 s, total: 3min 58s
# Wall time: 3min 58s

ais['h3_5'] = ais.apply(lambda row: h3.geo_to_h3( lat = row['lat'], lng = row['lon'], resolution = 5), axis = 1)
ais

CPU times: user 3min 42s, sys: 8.32 s, total: 3min 51s
Wall time: 3min 51s


,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,vessel_type,vessel_max_speed,draft,power,dwt,iceclass,h3_5
0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA,851f061bfffffff
1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA,851f061bfffffff
2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA,851f0613fffffff
3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA,851f0613fffffff
4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA,851f0613fffffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183139,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA,85088063fffffff
14183140,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA,8508806ffffffff
14183141,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA,8508806ffffffff
14183142,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,18.25,T,14.0,7.01,3350.0,7975.0,IA,8508806ffffffff


In [10]:
# ais.to_csv('data/ais.csv')

In [14]:
%%time 

# CPU times: user 7.08 s, sys: 2.97 s, total: 10.1 s
# Wall time: 10 s

ais_01_ports = ais.join(ports.set_index('h3_5'), on='h3_5')
ais_01_ports

CPU times: user 6.63 s, sys: 3.03 s, total: 9.66 s
Wall time: 9.65 s


,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,...,draft,power,dwt,iceclass,h3_5,name,port_id,lat_port,lon_port,iso2
0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,...,8.18,5820.0,13289.0,IA,851f061bfffffff,KIEL,111,54.3164,10.08312,de
1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,...,8.18,5820.0,13289.0,IA,851f061bfffffff,KIEL,111,54.3164,10.08312,de
2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,...,8.18,5820.0,13289.0,IA,851f0613fffffff,KIEL,111,54.3164,10.08312,de
3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,...,8.18,5820.0,13289.0,IA,851f0613fffffff,KIEL,111,54.3164,10.08312,de
4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,...,8.18,5820.0,13289.0,IA,851f0613fffffff,KIEL,111,54.3164,10.08312,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183139,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,18.25,...,7.01,3350.0,7975.0,IA,85088063fffffff,HALLSTAVIK,42199,60.0667,18.59900,se
14183140,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,18.25,...,7.01,3350.0,7975.0,IA,8508806ffffffff,Eckerö,747712,60.2290,19.54210,fi
14183141,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,18.25,...,7.01,3350.0,7975.0,IA,8508806ffffffff,Eckerö,747712,60.2290,19.54210,fi
14183142,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,18.25,...,7.01,3350.0,7975.0,IA,8508806ffffffff,Eckerö,747712,60.2290,19.54210,fi


In [15]:
%%time

# CPU times: user 25.6 s, sys: 5.93 s, total: 31.6 s
# Wall time: 31.6 s

ais_02_movement = ais_01_ports.drop_duplicates(subset=['mmsi', 'timestamp', 'lat', 'lon'], keep = 'first').reset_index(drop = True)
ais_02_movement

CPU times: user 25.7 s, sys: 5.62 s, total: 31.4 s
Wall time: 31.3 s


,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,...,draft,power,dwt,iceclass,h3_5,name,port_id,lat_port,lon_port,iso2
0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,...,8.18,5820.0,13289.0,IA,851f061bfffffff,KIEL,111,54.316400,10.083120,de
1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,...,8.18,5820.0,13289.0,IA,851f061bfffffff,KIEL,111,54.316400,10.083120,de
2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,...,8.18,5820.0,13289.0,IA,851f0613fffffff,KIEL,111,54.316400,10.083120,de
3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,...,8.18,5820.0,13289.0,IA,851f0613fffffff,KIEL,111,54.316400,10.083120,de
4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,...,8.18,5820.0,13289.0,IA,851f0613fffffff,KIEL,111,54.316400,10.083120,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183139,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,18.25,...,7.01,3350.0,7975.0,IA,85088063fffffff,GRISSLEHAMN,856529,60.110847,18.816747,se
14183140,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,18.25,...,7.01,3350.0,7975.0,IA,8508806ffffffff,Eckerö,747712,60.229000,19.542100,fi
14183141,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,18.25,...,7.01,3350.0,7975.0,IA,8508806ffffffff,Eckerö,747712,60.229000,19.542100,fi
14183142,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,18.25,...,7.01,3350.0,7975.0,IA,8508806ffffffff,Eckerö,747712,60.229000,19.542100,fi


In [17]:
%%time

# CPU times: user 29.9 s, sys: 2.28 s, total: 32.2 s
# Wall time: 32.2 s

ais_02_movement['in_port'] = np.where( ais_02_movement['port_id'].notnull(), 1, 0 )
ais_02_movement['lag_in_port'] = ais_02_movement.groupby(['mmsi'])['in_port'].shift(1)
ais_02_movement['in_port_gap'] = np.where( (ais_02_movement['lag_in_port'].notnull()) & (ais_02_movement['lag_in_port'] != ais_02_movement['in_port']), 1, 0 )
ais_02_movement['lag_port_id'] = ais_02_movement.groupby(['mmsi'])['port_id'].shift(1)
ais_02_movement['port_gap'] = np.where( (ais_02_movement['lag_port_id'].notnull()) & (ais_02_movement['lag_port_id'] != ais_02_movement['port_id']), 1, 0 )
ais_02_movement['stop_gap'] = np.where( (ais_02_movement['in_port_gap'] == 1) | (ais_02_movement['port_gap'] == 1), 1, 0 )
ais_02_movement['stop_movement'] = ais_02_movement.sort_values(by=['timestamp'], ascending=True).groupby(['mmsi'])['stop_gap'].cumsum()

ais_02_movement

CPU times: user 28.8 s, sys: 3.23 s, total: 32.1 s
Wall time: 32 s


,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,...,lat_port,lon_port,iso2,in_port,lag_in_port,in_port_gap,lag_port_id,port_gap,stop_gap,stop_movement
0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,...,54.316400,10.083120,de,1,NaN,0,NaN,0,0,0
1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,...,54.316400,10.083120,de,1,1.0,0,111,0,0,0
2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,...,54.316400,10.083120,de,1,1.0,0,111,0,0,0
3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,...,54.316400,10.083120,de,1,1.0,0,111,0,0,0
4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,...,54.316400,10.083120,de,1,1.0,0,111,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183139,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,18.25,...,60.110847,18.816747,se,1,1.0,0,747712,1,1,314
14183140,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,18.25,...,60.229000,19.542100,fi,1,1.0,0,856529,1,1,315
14183141,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,18.25,...,60.229000,19.542100,fi,1,1.0,0,747712,0,0,315
14183142,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,18.25,...,60.229000,19.542100,fi,1,1.0,0,747712,0,0,315


In [18]:
%%time

# CPU times: user 1min 36s, sys: 13 s, total: 1min 49s
# Wall time: 1min 49s

ais_03_movement = ais_02_movement.copy()

ais_03_movement['lag_lat'] = ais_03_movement.sort_values(by=['timestamp'], ascending=True).groupby(['mmsi','stop_movement'])['lat'].shift(1)
ais_03_movement['lag_lon'] = ais_03_movement.sort_values(by=['timestamp'], ascending=True).groupby(['mmsi','stop_movement'])['lon'].shift(1)
ais_03_movement['lag_timestamp'] = ais_03_movement.sort_values(by=['timestamp'], ascending=True).groupby(['mmsi','stop_movement'])['timestamp'].shift(1)

ais_03_movement['time_interval']  = (pd.to_datetime(ais_03_movement['timestamp']) - pd.to_datetime(ais_03_movement['lag_timestamp'])).astype('timedelta64[s]')
ais_03_movement['time_interval_hour'] = np.round( ais_03_movement['time_interval'] / 3600 , decimals=2)

ais_03_movement

CPU times: user 1min 35s, sys: 12.8 s, total: 1min 48s
Wall time: 1min 47s


,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,...,in_port_gap,lag_port_id,port_gap,stop_gap,stop_movement,lag_lat,lag_lon,lag_timestamp,time_interval,time_interval_hour
0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,...,0,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,...,0,111,0,0,0,54.347242,9.991140,2017-11-02 11:19:07,711.0,0.20
2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,...,0,111,0,0,0,54.360348,10.025337,2017-11-02 11:30:58,420.0,0.12
3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,...,0,111,0,0,0,54.359573,10.049203,2017-11-02 11:37:58,369.0,0.10
4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,...,0,111,0,0,0,54.365497,10.071320,2017-11-02 11:44:07,721.0,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183139,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,18.25,...,0,747712,1,1,314,NaN,NaN,NaN,NaN,NaN
14183140,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,18.25,...,0,856529,1,1,315,NaN,NaN,NaN,NaN,NaN
14183141,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,18.25,...,0,747712,0,0,315,60.116143,19.129962,2018-08-16 15:13:22,610.0,0.17
14183142,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,18.25,...,0,747712,0,0,315,60.088740,19.178022,2018-08-16 15:23:32,569.0,0.16


In [21]:
def haversine(lon1, lat1, lon2, lat2):                                              
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    a = np.sin((lat2 - lat1) / 2.0)**2 + (np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2.0)**2)
    distance = 6371 * 2 * np.arcsin(np.sqrt(a))
    return distance

def speed(distance, duration):
    time = duration / 3600
    speed = distance / time
    return speed

In [23]:
%%time

# CPU times: user 1.41 s, sys: 673 ms, total: 2.09 s
# Wall time: 2.09 s

ais_03_movement['distance'] = haversine(ais_03_movement['lon'], ais_03_movement['lat'],
                                        ais_03_movement['lag_lon'], ais_03_movement['lag_lat'])
ais_03_movement

CPU times: user 1.41 s, sys: 673 ms, total: 2.09 s
Wall time: 2.09 s


,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,...,lag_port_id,port_gap,stop_gap,stop_movement,lag_lat,lag_lon,lag_timestamp,time_interval,time_interval_hour,distance
0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,...,111,0,0,0,54.347242,9.991140,2017-11-02 11:19:07,711.0,0.20,2.652303
2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,...,111,0,0,0,54.360348,10.025337,2017-11-02 11:30:58,420.0,0.12,1.548775
3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,...,111,0,0,0,54.359573,10.049203,2017-11-02 11:37:58,369.0,0.10,1.577027
4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,...,111,0,0,0,54.365497,10.071320,2017-11-02 11:44:07,721.0,0.20,2.533446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183139,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,18.25,...,747712,1,1,314,NaN,NaN,NaN,NaN,NaN,NaN
14183140,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,18.25,...,856529,1,1,315,NaN,NaN,NaN,NaN,NaN,NaN
14183141,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,18.25,...,747712,0,0,315,60.116143,19.129962,2018-08-16 15:13:22,610.0,0.17,4.047267
14183142,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,18.25,...,747712,0,0,315,60.088740,19.178022,2018-08-16 15:23:32,569.0,0.16,3.784783


In [24]:
%%time

# CPU times: user 105 ms, sys: 80.2 ms, total: 185 ms
# Wall time: 184 ms

ais_03_movement['speed_kmh'] = speed(ais_03_movement['distance'], ais_03_movement['time_interval'])
ais_03_movement

CPU times: user 105 ms, sys: 80.2 ms, total: 185 ms
Wall time: 184 ms


,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,...,port_gap,stop_gap,stop_movement,lag_lat,lag_lon,lag_timestamp,time_interval,time_interval_hour,distance,speed_kmh
0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,...,0,0,0,54.347242,9.991140,2017-11-02 11:19:07,711.0,0.20,2.652303,13.429383
2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,...,0,0,0,54.360348,10.025337,2017-11-02 11:30:58,420.0,0.12,1.548775,13.275216
3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,...,0,0,0,54.359573,10.049203,2017-11-02 11:37:58,369.0,0.10,1.577027,15.385629
4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,...,0,0,0,54.365497,10.071320,2017-11-02 11:44:07,721.0,0.20,2.533446,12.649662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183139,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,18.25,...,1,1,314,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14183140,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,18.25,...,1,1,315,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14183141,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,18.25,...,0,0,315,60.116143,19.129962,2018-08-16 15:13:22,610.0,0.17,4.047267,23.885512
14183142,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,18.25,...,0,0,315,60.088740,19.178022,2018-08-16 15:23:32,569.0,0.16,3.784783,23.945901


In [5]:
# ais_03_movement.to_csv('data/ais_03_movement.csv')

# ais_03_movement = pd.read_csv('data/ais_03_movement.csv')
# ais_03_movement

/tmp/ipykernel_11580/359902757.py:3: DtypeWarning: Columns (16,18,19,22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  ais_03_movement = pd.read_csv('data/ais_03_movement.csv')


,Unnamed: 0.1,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,...,port_gap,stop_gap,stop_movement,lag_lat,lag_lon,lag_timestamp,time_interval,time_interval_hour,distance,speed_kmh
0,0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,...,0,0,0,54.347242,9.991140,2017-11-02 11:19:07,711.0,0.20,2.652303,13.429383
2,2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,...,0,0,0,54.360348,10.025337,2017-11-02 11:30:58,420.0,0.12,1.548775,13.275216
3,3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,...,0,0,0,54.359573,10.049203,2017-11-02 11:37:58,369.0,0.10,1.577027,15.385629
4,4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,...,0,0,0,54.365497,10.071320,2017-11-02 11:44:07,721.0,0.20,2.533446,12.649662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183139,14183139,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,...,1,1,314,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14183140,14183140,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,...,1,1,315,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14183141,14183141,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,...,0,0,315,60.116143,19.129962,2018-08-16 15:13:22,610.0,0.17,4.047267,23.885512
14183142,14183142,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,...,0,0,315,60.088740,19.178022,2018-08-16 15:23:32,569.0,0.16,3.784783,23.945901


In [6]:
%%time

# CPU times: user 19.6 s, sys: 3.59 s, total: 23.2 s
# Wall time: 23.2 s

ais_03_movement['start_timestamp'] = ais_03_movement.groupby(['mmsi','stop_movement'])['timestamp'].transform('first')
ais_03_movement['end_timestamp']   = ais_03_movement.groupby(['mmsi','stop_movement'])['timestamp'].transform('last')
ais_03_movement['min_speed']   = ais_03_movement.groupby(['mmsi','stop_movement'])['speed_kmh'].transform('min')
ais_03_movement['avg_speed'] = ais_03_movement.groupby(['mmsi','stop_movement'])['speed_kmh'].transform(np.mean)
ais_03_movement['max_speed']   = ais_03_movement.groupby(['mmsi','stop_movement'])['speed_kmh'].transform('max')
ais_03_movement['sum_distance']   = ais_03_movement.groupby(['mmsi','stop_movement'])['distance'].transform('sum')
ais_03_movement['coordinate'] = ais_03_movement[['lon','lat']].values.tolist()

ais_03_movement

CPU times: user 19.6 s, sys: 3.59 s, total: 23.2 s
Wall time: 23.2 s


,Unnamed: 0.1,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,...,time_interval_hour,distance,speed_kmh,start_timestamp,end_timestamp,min_speed,avg_speed,max_speed,sum_distance,coordinate
0,0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,...,NaN,NaN,NaN,2017-11-02 11:19:07,2017-11-02 13:15:19,0.053979,9.530489,18.095926,16.096867,"[9.99114, 54.34724166666667]"
1,1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,...,0.20,2.652303,13.429383,2017-11-02 11:19:07,2017-11-02 13:15:19,0.053979,9.530489,18.095926,16.096867,"[10.025336666666666, 54.36034833333333]"
2,2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,...,0.12,1.548775,13.275216,2017-11-02 11:19:07,2017-11-02 13:15:19,0.053979,9.530489,18.095926,16.096867,"[10.049203333333333, 54.35957333333333]"
3,3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,...,0.10,1.577027,15.385629,2017-11-02 11:19:07,2017-11-02 13:15:19,0.053979,9.530489,18.095926,16.096867,"[10.07132, 54.36549666666666]"
4,4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,...,0.20,2.533446,12.649662,2017-11-02 11:19:07,2017-11-02 13:15:19,0.053979,9.530489,18.095926,16.096867,"[10.109498333333333, 54.37043666666667]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183139,14183139,14185041,2018-08-16 15:03:22,257222000,60.143353,19.082380,6.64,138.1,141.0,0.0,...,NaN,NaN,NaN,2018-08-16 15:03:22,2018-08-16 15:03:22,NaN,NaN,NaN,0.000000,"[19.08238, 60.14335333333333]"
14183140,14183140,14185042,2018-08-16 15:13:22,257222000,60.116143,19.129962,6.64,138.9,142.0,0.0,...,NaN,NaN,NaN,2018-08-16 15:13:22,2018-08-16 15:43:02,23.885512,23.930428,23.959871,11.832017,"[19.129961666666667, 60.11614333333333]"
14183141,14183141,14185043,2018-08-16 15:23:32,257222000,60.088740,19.178022,6.53,138.8,142.0,0.0,...,0.17,4.047267,23.885512,2018-08-16 15:13:22,2018-08-16 15:43:02,23.885512,23.930428,23.959871,11.832017,"[19.178021666666663, 60.08874]"
14183142,14183142,14185044,2018-08-16 15:33:01,257222000,60.062992,19.222647,6.64,140.6,144.0,0.0,...,0.16,3.784783,23.945901,2018-08-16 15:13:22,2018-08-16 15:43:02,23.885512,23.930428,23.959871,11.832017,"[19.222646666666662, 60.06299166666667]"


In [32]:
%%time

# CPU times: user 9.67 s, sys: 3.61 s, total: 13.3 s
# Wall time: 13.3 s

ais_04_movement = ais_03_movement.copy()
ais_04_movement = ais_04_movement.drop_duplicates(subset=['mmsi', 'start_timestamp', 'end_timestamp'], keep = 'first').reset_index(drop = True)

ais_04_movement =  ais_04_movement[[ 'mmsi',  'vessel_type',
                        'h3_5', 'name', 'port_id', 'lat_port', 'lon_port', 'iso2', 
                        'start_timestamp', 'end_timestamp', 'min_speed', 
                        'avg_speed', 'max_speed', 'sum_distance',
                        'coordinate']]

ais_04_movement

CPU times: user 9.67 s, sys: 3.61 s, total: 13.3 s
Wall time: 13.3 s


,mmsi,vessel_type,h3_5,name,port_id,lat_port,lon_port,iso2,start_timestamp,end_timestamp,min_speed,avg_speed,max_speed,sum_distance,coordinate
0,205366000,T,851f061bfffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-02 11:19:07,2017-11-02 13:15:19,0.053979,9.530489,18.095926,16.096867,"[9.99114, 54.34724166666667]"
1,205366000,T,851f0617fffffff,NaN,NaN,NaN,NaN,NaN,2017-11-02 13:33:33,2017-11-04 05:54:02,0.001785,22.067559,33.320728,666.394033,"[10.24836, 54.45144166666667]"
2,205366000,T,851f061bfffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-17 03:24:02,2017-11-17 05:08:16,0.192683,7.405207,14.396141,10.508815,"[9.992045, 54.34755833333333]"
3,205366000,T,851f0617fffffff,NaN,NaN,NaN,NaN,NaN,2017-11-17 05:41:58,2017-11-19 08:07:28,0.001644,9.930060,33.236793,310.360159,"[10.223201666666666, 54.42488166666666]"
4,205366000,T,851f0603fffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-19 08:19:38,2017-11-24 18:48:22,0.011750,7.506310,17.962700,30.640641,"[10.200446666666666, 54.400325]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284093,257222000,T,8508aad3fffffff,SÖDERHAMN,12812.0,61.314200,17.081200,se,2018-08-16 06:43:40,2018-08-16 07:42:01,11.180884,22.213147,27.872938,21.991128,"[17.473391666666668, 61.451798333333336]"
284094,257222000,T,85088537fffffff,NaN,NaN,NaN,NaN,NaN,2018-08-16 07:52:12,2018-08-16 14:23:41,19.828226,24.057768,27.009876,157.595796,"[17.5866, 61.25617833333333]"
284095,257222000,T,85088067fffffff,Eckerö,747712.0,60.229000,19.542100,fi,2018-08-16 14:33:00,2018-08-16 14:49:11,19.781796,21.845558,23.909320,6.025804,"[19.002563333333327, 60.236988333333336]"
284096,257222000,T,85088063fffffff,GRISSLEHAMN,856529.0,60.110847,18.816747,se,2018-08-16 15:03:22,2018-08-16 15:03:22,NaN,NaN,NaN,0.000000,"[19.08238, 60.14335333333333]"


In [33]:
# ais_04_movement.to_csv('data/ais_04_movement.csv')

# ais_04_movement = pd.read_csv('data/ais_04_movement.csv')
# ais_04_movement

In [34]:
%%time

# CPU times: user 2.46 s, sys: 80.2 ms, total: 2.54 s
# Wall time: 2.54 s

ais_05_shipping = ais_04_movement.copy()

ais_05_shipping['lag_start_timestamp'] = ais_05_shipping.sort_values(by=['start_timestamp'], ascending=True).groupby(['mmsi'])['start_timestamp'].shift(1)
ais_05_shipping['lag_end_timestamp'] = ais_05_shipping.sort_values(by=['start_timestamp'], ascending=True).groupby(['mmsi'])['end_timestamp'].shift(1)
ais_05_shipping['lead_start_timestamp'] = ais_05_shipping.sort_values(by=['start_timestamp'], ascending=True).groupby(['mmsi'])['start_timestamp'].shift(-1)
ais_05_shipping['lead_end_timestamp'] = ais_05_shipping.sort_values(by=['start_timestamp'], ascending=True).groupby(['mmsi'])['end_timestamp'].shift(-1)

ais_05_shipping

CPU times: user 2.46 s, sys: 80.2 ms, total: 2.54 s
Wall time: 2.54 s


,mmsi,vessel_type,h3_5,name,port_id,lat_port,lon_port,iso2,start_timestamp,end_timestamp,min_speed,avg_speed,max_speed,sum_distance,coordinate,lag_start_timestamp,lag_end_timestamp,lead_start_timestamp,lead_end_timestamp
0,205366000,T,851f061bfffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-02 11:19:07,2017-11-02 13:15:19,0.053979,9.530489,18.095926,16.096867,"[9.99114, 54.34724166666667]",NaN,NaN,2017-11-02 13:33:33,2017-11-04 05:54:02
1,205366000,T,851f0617fffffff,NaN,NaN,NaN,NaN,NaN,2017-11-02 13:33:33,2017-11-04 05:54:02,0.001785,22.067559,33.320728,666.394033,"[10.24836, 54.45144166666667]",2017-11-02 11:19:07,2017-11-02 13:15:19,2017-11-17 03:24:02,2017-11-17 05:08:16
2,205366000,T,851f061bfffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-17 03:24:02,2017-11-17 05:08:16,0.192683,7.405207,14.396141,10.508815,"[9.992045, 54.34755833333333]",2017-11-02 13:33:33,2017-11-04 05:54:02,2017-11-17 05:41:58,2017-11-19 08:07:28
3,205366000,T,851f0617fffffff,NaN,NaN,NaN,NaN,NaN,2017-11-17 05:41:58,2017-11-19 08:07:28,0.001644,9.930060,33.236793,310.360159,"[10.223201666666666, 54.42488166666666]",2017-11-17 03:24:02,2017-11-17 05:08:16,2017-11-19 08:19:38,2017-11-24 18:48:22
4,205366000,T,851f0603fffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-19 08:19:38,2017-11-24 18:48:22,0.011750,7.506310,17.962700,30.640641,"[10.200446666666666, 54.400325]",2017-11-17 05:41:58,2017-11-19 08:07:28,2017-11-24 18:54:31,2017-11-26 02:36:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284093,257222000,T,8508aad3fffffff,SÖDERHAMN,12812.0,61.314200,17.081200,se,2018-08-16 06:43:40,2018-08-16 07:42:01,11.180884,22.213147,27.872938,21.991128,"[17.473391666666668, 61.451798333333336]",2018-08-16 06:16:51,2018-08-16 06:33:40,2018-08-16 07:52:12,2018-08-16 14:23:41
284094,257222000,T,85088537fffffff,NaN,NaN,NaN,NaN,NaN,2018-08-16 07:52:12,2018-08-16 14:23:41,19.828226,24.057768,27.009876,157.595796,"[17.5866, 61.25617833333333]",2018-08-16 06:43:40,2018-08-16 07:42:01,2018-08-16 14:33:00,2018-08-16 14:49:11
284095,257222000,T,85088067fffffff,Eckerö,747712.0,60.229000,19.542100,fi,2018-08-16 14:33:00,2018-08-16 14:49:11,19.781796,21.845558,23.909320,6.025804,"[19.002563333333327, 60.236988333333336]",2018-08-16 07:52:12,2018-08-16 14:23:41,2018-08-16 15:03:22,2018-08-16 15:03:22
284096,257222000,T,85088063fffffff,GRISSLEHAMN,856529.0,60.110847,18.816747,se,2018-08-16 15:03:22,2018-08-16 15:03:22,NaN,NaN,NaN,0.000000,"[19.08238, 60.14335333333333]",2018-08-16 14:33:00,2018-08-16 14:49:11,2018-08-16 15:13:22,2018-08-16 15:43:02


In [35]:
%%time

# CPU times: user 1.04 s, sys: 19.6 ms, total: 1.06 s
# Wall time: 1.06 s

ais_05_shipping['lag_time_interval']  = (pd.to_datetime(ais_05_shipping['start_timestamp']) - pd.to_datetime(ais_05_shipping['lag_end_timestamp'])).astype('timedelta64[s]')
ais_05_shipping['lead_time_interval']  = (pd.to_datetime(ais_05_shipping['lead_start_timestamp']) - pd.to_datetime(ais_05_shipping['end_timestamp'])).astype('timedelta64[s]')

ais_05_shipping['shipping_status'] = np.where(ais_05_shipping['lag_time_interval'].isnull(), 0 ,
                                        np.where (ais_05_shipping['lag_time_interval'] > 10000, 0, 
                                            np.where(ais_05_shipping['lead_time_interval'].isnull(), 2, 
                                                np.where(ais_05_shipping['lead_time_interval'] > 10000, 2, 
                                                    1 ))))
ais_05_shipping['shipping_gap'] = np.where( ais_05_shipping['shipping_status'] == 0, 1, 0 )
ais_05_shipping['shipping_movement'] = ais_05_shipping.sort_values(by=['start_timestamp'], ascending=True).groupby(['mmsi'])['shipping_gap'].cumsum()

ais_05_shipping

CPU times: user 1.04 s, sys: 19.6 ms, total: 1.06 s
Wall time: 1.06 s


,mmsi,vessel_type,h3_5,name,port_id,lat_port,lon_port,iso2,start_timestamp,end_timestamp,...,coordinate,lag_start_timestamp,lag_end_timestamp,lead_start_timestamp,lead_end_timestamp,lag_time_interval,lead_time_interval,shipping_status,shipping_gap,shipping_movement
0,205366000,T,851f061bfffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-02 11:19:07,2017-11-02 13:15:19,...,"[9.99114, 54.34724166666667]",NaN,NaN,2017-11-02 13:33:33,2017-11-04 05:54:02,NaN,1094.0,0,1,1
1,205366000,T,851f0617fffffff,NaN,NaN,NaN,NaN,NaN,2017-11-02 13:33:33,2017-11-04 05:54:02,...,"[10.24836, 54.45144166666667]",2017-11-02 11:19:07,2017-11-02 13:15:19,2017-11-17 03:24:02,2017-11-17 05:08:16,1094.0,1114200.0,2,0,1
2,205366000,T,851f061bfffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-17 03:24:02,2017-11-17 05:08:16,...,"[9.992045, 54.34755833333333]",2017-11-02 13:33:33,2017-11-04 05:54:02,2017-11-17 05:41:58,2017-11-19 08:07:28,1114200.0,2022.0,0,1,2
3,205366000,T,851f0617fffffff,NaN,NaN,NaN,NaN,NaN,2017-11-17 05:41:58,2017-11-19 08:07:28,...,"[10.223201666666666, 54.42488166666666]",2017-11-17 03:24:02,2017-11-17 05:08:16,2017-11-19 08:19:38,2017-11-24 18:48:22,2022.0,730.0,1,0,2
4,205366000,T,851f0603fffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-19 08:19:38,2017-11-24 18:48:22,...,"[10.200446666666666, 54.400325]",2017-11-17 05:41:58,2017-11-19 08:07:28,2017-11-24 18:54:31,2017-11-26 02:36:10,730.0,369.0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284093,257222000,T,8508aad3fffffff,SÖDERHAMN,12812.0,61.314200,17.081200,se,2018-08-16 06:43:40,2018-08-16 07:42:01,...,"[17.473391666666668, 61.451798333333336]",2018-08-16 06:16:51,2018-08-16 06:33:40,2018-08-16 07:52:12,2018-08-16 14:23:41,600.0,611.0,1,0,4
284094,257222000,T,85088537fffffff,NaN,NaN,NaN,NaN,NaN,2018-08-16 07:52:12,2018-08-16 14:23:41,...,"[17.5866, 61.25617833333333]",2018-08-16 06:43:40,2018-08-16 07:42:01,2018-08-16 14:33:00,2018-08-16 14:49:11,611.0,559.0,1,0,4
284095,257222000,T,85088067fffffff,Eckerö,747712.0,60.229000,19.542100,fi,2018-08-16 14:33:00,2018-08-16 14:49:11,...,"[19.002563333333327, 60.236988333333336]",2018-08-16 07:52:12,2018-08-16 14:23:41,2018-08-16 15:03:22,2018-08-16 15:03:22,559.0,851.0,1,0,4
284096,257222000,T,85088063fffffff,GRISSLEHAMN,856529.0,60.110847,18.816747,se,2018-08-16 15:03:22,2018-08-16 15:03:22,...,"[19.08238, 60.14335333333333]",2018-08-16 14:33:00,2018-08-16 14:49:11,2018-08-16 15:13:22,2018-08-16 15:43:02,851.0,600.0,1,0,4


In [36]:
# ais_05_shipping.to_csv('data/ais_05_shipping.csv')

# ais_05_shipping = pd.read_csv('data/ais_05_shipping.csv')
# ais_05_shipping

In [69]:
%%time

ais_06_shipping = ais_05_shipping.copy()

ais_06_shipping['id_shipping'] = ais_06_shipping['mmsi'].map(str) + '-' + ais_06_shipping['shipping_movement'].map(str)

ais_06_shipping['start_time'] = ais_06_shipping.groupby(['id_shipping'])['start_timestamp'].transform('first')
ais_06_shipping['end_time']   = ais_06_shipping.groupby(['id_shipping'])['end_timestamp'].transform('last')

ais_06_shipping['origin_port_id'] = ais_06_shipping.groupby(['id_shipping'])['port_id'].transform('first')
ais_06_shipping['origin_port_name'] = ais_06_shipping.groupby(['id_shipping'])['name'].transform('first')
ais_06_shipping['origin_country_iso2'] = ais_06_shipping.groupby(['id_shipping'])['iso2'].transform('first')

ais_06_shipping['destination_port_id'] = ais_06_shipping.groupby(['id_shipping'])['port_id'].transform('last')
ais_06_shipping['destination_port_name'] = ais_06_shipping.groupby(['id_shipping'])['name'].transform('last')
ais_06_shipping['destination_country_iso2'] = ais_06_shipping.groupby(['id_shipping'])['iso2'].transform('last')

ais_06_shipping['avg_speed']   = ais_06_shipping.groupby(['id_shipping'])['avg_speed'].transform(np.mean)
ais_06_shipping['distance']   = ais_06_shipping.groupby(['id_shipping'])['sum_distance'].transform('sum')

ais_06_shipping

CPU times: user 1.27 s, sys: 136 ms, total: 1.4 s
Wall time: 1.4 s


,mmsi,vessel_type,h3_5,name,port_id,lat_port,lon_port,iso2,start_timestamp,end_timestamp,...,id_shipping,start_time,end_time,origin_port_id,origin_port_name,origin_country_iso2,destination_port_id,destination_port_name,destination_country_iso2,distance
0,205366000,T,851f061bfffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-02 11:19:07,2017-11-02 13:15:19,...,205366000-1,2017-11-02 11:19:07,2017-11-04 05:54:02,111.0,KIEL,de,111.0,KIEL,de,682.490900
1,205366000,T,851f0617fffffff,NaN,NaN,NaN,NaN,NaN,2017-11-02 13:33:33,2017-11-04 05:54:02,...,205366000-1,2017-11-02 11:19:07,2017-11-04 05:54:02,111.0,KIEL,de,111.0,KIEL,de,682.490900
2,205366000,T,851f061bfffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-17 03:24:02,2017-11-17 05:08:16,...,205366000-2,2017-11-17 03:24:02,2017-12-22 21:05:04,111.0,KIEL,de,350.0,SIMRISHAMN,se,2616.645503
3,205366000,T,851f0617fffffff,NaN,NaN,NaN,NaN,NaN,2017-11-17 05:41:58,2017-11-19 08:07:28,...,205366000-2,2017-11-17 03:24:02,2017-12-22 21:05:04,111.0,KIEL,de,350.0,SIMRISHAMN,se,2616.645503
4,205366000,T,851f0603fffffff,KIEL,111.0,54.316400,10.083120,de,2017-11-19 08:19:38,2017-11-24 18:48:22,...,205366000-2,2017-11-17 03:24:02,2017-12-22 21:05:04,111.0,KIEL,de,350.0,SIMRISHAMN,se,2616.645503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284093,257222000,T,8508aad3fffffff,SÖDERHAMN,12812.0,61.314200,17.081200,se,2018-08-16 06:43:40,2018-08-16 07:42:01,...,257222000-4,2018-07-19 13:06:39,2018-08-16 15:43:02,111.0,KIEL,de,747712.0,Eckerö,fi,6519.351639
284094,257222000,T,85088537fffffff,NaN,NaN,NaN,NaN,NaN,2018-08-16 07:52:12,2018-08-16 14:23:41,...,257222000-4,2018-07-19 13:06:39,2018-08-16 15:43:02,111.0,KIEL,de,747712.0,Eckerö,fi,6519.351639
284095,257222000,T,85088067fffffff,Eckerö,747712.0,60.229000,19.542100,fi,2018-08-16 14:33:00,2018-08-16 14:49:11,...,257222000-4,2018-07-19 13:06:39,2018-08-16 15:43:02,111.0,KIEL,de,747712.0,Eckerö,fi,6519.351639
284096,257222000,T,85088063fffffff,GRISSLEHAMN,856529.0,60.110847,18.816747,se,2018-08-16 15:03:22,2018-08-16 15:03:22,...,257222000-4,2018-07-19 13:06:39,2018-08-16 15:43:02,111.0,KIEL,de,747712.0,Eckerö,fi,6519.351639


In [70]:
ais_06_shipping = ais_06_shipping.groupby(['id_shipping','mmsi', 'vessel_type', 'start_time', 'end_time', 
        'origin_port_id', 'origin_port_name', 'origin_country_iso2', 
        'destination_port_id', 'destination_port_name','destination_country_iso2', 'avg_speed', 'distance'])['coordinate'].apply(list).reset_index(name='movement_coordinates')

ais_06_shipping = ais_06_shipping.drop_duplicates(subset=['id_shipping'], keep = 'first').reset_index(drop = True)

ais_06_shipping

,id_shipping,mmsi,vessel_type,start_time,end_time,origin_port_id,origin_port_name,origin_country_iso2,destination_port_id,destination_port_name,destination_country_iso2,avg_speed,distance,movement_coordinates
0,205366000-1,205366000,T,2017-11-02 11:19:07,2017-11-04 05:54:02,111.0,KIEL,de,111.0,KIEL,de,15.799024,682.490900,"[[9.99114, 54.34724166666667], [10.24836, 54.4..."
1,205366000-2,205366000,T,2017-11-17 03:24:02,2017-12-22 21:05:04,111.0,KIEL,de,350.0,SIMRISHAMN,se,17.522296,2616.645503,"[[9.992045, 54.34755833333333], [10.2232016666..."
2,205366000-3,205366000,T,2017-12-28 05:05:57,2018-01-14 20:26:02,111.0,KIEL,de,111.0,KIEL,de,13.561618,1029.175079,"[[10.01526, 54.35835], [10.206488333333333, 54..."
3,205366000-4,205366000,T,2018-05-18 16:19:17,2018-06-03 16:53:49,111.0,KIEL,de,111.0,KIEL,de,11.723741,1044.251382,"[[9.993935, 54.34839333333333], [10.2207883333..."
4,205366000-5,205366000,T,2018-06-07 19:49:26,2018-06-07 20:49:16,111.0,KIEL,de,111.0,KIEL,de,9.899382,10.212245,"[[9.986568333333333, 54.34567833333333]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5555,257207000-4,257207000,GC,2018-09-20 18:40:30,2018-10-21 17:21:46,9217.0,LÄNSI,no,782598.0,TRELLEBORG,se,18.721504,2264.732040,"[[10.55196, 59.43799], [10.600216666666666, 59..."
5556,257222000-1,257222000,T,2017-11-01 09:08:43,2018-05-22 11:25:37,747712.0,Eckerö,fi,32140.0,HÖGANÄS,se,20.087964,22522.148706,"[[21.71331, 64.01657333333333], [18.999215, 60..."
5557,257222000-2,257222000,T,2018-06-01 04:51:30,2018-06-10 15:31:26,111.0,KIEL,de,350.0,SIMRISHAMN,se,20.142835,3830.454914,"[[9.99738, 54.35017166666667], [10.21074666666..."
5558,257222000-3,257222000,T,2018-06-18 09:31:05,2018-07-14 22:42:49,111.0,KIEL,de,350.0,SIMRISHAMN,se,18.017888,6683.691764,"[[9.991855, 54.347475], [10.216175, 54.4175550..."


In [71]:
ais_06_shipping.to_csv('data/ais_06_shipping.csv')

# ais_06_shipping = pd.read_csv('data/ais_06_shipping.csv')
# ais_06_shipping